## Phi-4 RAG mit Azure AI Search

Dieses Notebook zeigt, wie Phi-4-mini und Phi-4-multimodal für Retrieval Augmented Generation (RAG) mit Azure AI Search verwendet werden können. Es behandelt sowohl Einzelmodalität (nur Text) als auch Multimodalität (Text und Bild) Szenarien.

**Voraussetzungen:**
*   Azure AI Search Vektorindex (befolgen Sie [diese Anleitung](https://learn.microsoft.com/azure/search/search-get-started-portal-import-vectors?tabs=sample-data-storage%2Cmodel-aoai%2Cconnect-data-storage), um einen zu erstellen)
*   Phi-4-mini ODER Phi-4-multimodal Endpunkte, die in Azure AI Foundry bereitgestellt wurden


In [ ]:
! pip install azure-ai-inference azure-search-documents

### Text-Only RAG mit Phi-4-mini

Dieser Abschnitt zeigt, wie Phi-4-mini als Chat-Completion-Modell für RAG verwendet werden kann, wobei ausschließlich Text als Eingabe dient. Es umfasst die Verbindung zu Azure AI Foundry Inference und Azure AI Search, das Abrufen relevanter Dokumente und das Generieren einer Antwort mithilfe des abgerufenen Kontexts.


In [ ]:
# Step 1: Connect to Azure AI Foundry Inference & Azure AI Search
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], # Phi-4-mini endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 10):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query,vector_queries=[vector_query],select=["content"],top=top)
    return [doc["content"] for doc in results]

# Step 3: Generate answer using RAG!
def generate_rag_response(query: str):
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    response = chat_client.complete(messages=[UserMessage(content=prompt)])
    return response.choices[0].message.content

# Example usage
user_query = "What is the capital of France?"
answer = generate_rag_response(user_query)
print(f"Q: {user_query}\nA: {answer}")


### Multi-Modal RAG mit Phi-4-multimodal

Dieser Abschnitt zeigt, wie Phi-4-multimodal als Chat-Completion-Modell für RAG verwendet werden kann, indem sowohl Text- als auch Bildinput integriert werden. Es behandelt die Verbindung zu Azure AI Inference und Azure AI Search, das Abrufen relevanter Dokumente und das Erstellen einer multimodalen Antwort.

**Hinweis:** Sie können auch eine Multi-Vektor-Abfrage durchführen, wenn Sie sowohl `text_vector`- als auch `image_vector`-Felder in Ihrem Azure AI Search-Index haben.


In [ ]:
# Step 1: Connect to Azure AI Inference & Azure AI Search
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery


chat_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT"], #Phi-4-multimodal serverless endpoint
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL"]),
)

search_client = SearchClient(
    endpoint=os.environ["AZURE_SEARCH_ENDPOINT"],
    index_name=os.environ["AZURE_SEARCH_INDEX"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_KEY"])
)

# Step 2: Retrieve relevant documents from Azure AI Search
def retrieve_documents(query: str, top: int = 3):
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=top, fields="text_vector")
    results = search_client.search(search_text=query, vector_queries=[vector_query], select=["content"], top=top)    
    return [doc["content"] for doc in results]

## Example for Muli-modal search if you have a text_vector AND image_vector field in your vector_index
## NOTE, image vectorization is in preview at the time of writing this code, please use azure-search-documents pypi version >11.6.0b6 
# def retrieve_documents_multimodal(query: str, image_url: str, top: int = 3):
#     text_vector_query = VectorizableTextQuery(
#         text=query,
#         k_nearest_neighbors=top,
#         fields="text_vector",
#         weight=0.5  # Adjust weight as needed
#     )
#     image_vector_query = VectorizableImageUrlQuery(
#         url=image_url,
#         k_nearest_neighbors=top,
#         fields="image_vector",
#         weight=0.5  # Adjust weight as needed
#     )

#     results = search_client.search(
#         search_text=query,  
#         vector_queries=[text_vector_query, image_vector_query],
#         select=["content"],
#         top=top
#     )
#     return [doc["content"] for doc in results]


# Step 3: Generate a multimodal RAG-based answer using retrieved text and an image input
def generate_multimodal_rag_response(query: str, image_url: str):
    # Retrieve text context from search
    docs = retrieve_documents(query)
    context = "\n---\n".join(docs)

    # Build a prompt that combines the retrieved context with the user query
    prompt = f"""You are a helpful assistant. Use only the following context to answer the question. If the answer isn't in the context, say 'I don't know'.
    Context: {context} Question: {query} Answer:"""
    # Create a chat request that includes both text and image input
    response = chat_client.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant that can process both text and images."),
            UserMessage(
                content=[
                    TextContentItem(text=prompt),
                    ImageContentItem(image_url=ImageUrl(url=image_url, detail=ImageDetailLevel.HIGH)),
                ]
            ),
        ]
    )
    return response.choices[0].message.content

# Example usage
user_query = "Can you search for similar items to this shoe?"
sample_image_url = "https://images.unsplash.com/photo-1542291026-7eec264c27ff?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3"
answer = generate_multimodal_rag_response(user_query, sample_image_url)
print(f"Q: {user_query}\nA: {answer}")


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
